In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heartbeat/ptbdb_abnormal.csv
/kaggle/input/heartbeat/ptbdb_normal.csv
/kaggle/input/heartbeat/mitbih_test.csv
/kaggle/input/heartbeat/mitbih_train.csv


In [2]:
import matplotlib.pyplot as plt

import pywt
import scipy
from skimage.restoration import denoise_wavelet

import tensorflow as tf
from tensorflow import keras
from sklearn.cluster import KMeans
from tensorflow.keras.layers import Layer, Dense, Dropout, BatchNormalization
from tensorflow.keras.initializers import RandomUniform, Initializer, Constant
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report

from keras.utils.np_utils import to_categorical 

from keras.optimizers import RMSprop,Adam
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau

In [3]:
train_df=pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv',header=None)
test_df=pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv',header=None)

In [4]:
train_x = np.array(train_df[train_df.columns[0:-1]], dtype=np.float32)
train_y = np.array(train_df[train_df.columns[-1]], dtype=np.float32)

test_x = np.array(test_df[test_df.columns[0:-1]], dtype=np.float32)
test_y = np.array(test_df[test_df.columns[-1]], dtype=np.float32)

In [5]:
c=train_df.groupby(187,group_keys=False).apply(lambda train_df : train_df.sample(1))
cind=c.index.to_numpy()
cind

array([56376, 73329, 78752, 80812, 82784])

In [6]:
for i in range(len(train_x)):
    train_x[i] =  denoise_wavelet(train_x[i] ,method='BayesShrink', mode='soft',
                           rescale_sigma=True, wavelet='sym8', wavelet_levels=3)

for i in range(len(test_x)):
    test_x[i] =  denoise_wavelet(test_x[i] ,method='BayesShrink', mode='soft',
                           rescale_sigma=True, wavelet='sym8', wavelet_levels=3)

In [7]:
def wpd(x):

    Xfeatures=np.zeros((len(x),8,29))

    for i in range(len(x)):
        wpack=pywt.WaveletPacket(data=x[i], wavelet='db4')
        Xfeatures[i]=np.vstack((wpack['aaa'].data,wpack['add'].data,wpack['dad'].data,wpack['dda'].data,
                                wpack['aad'].data,wpack['daa'].data,wpack['ada'].data,wpack['ddd'].data))
    
    return Xfeatures

In [8]:
Xfeatures=wpd(train_x)

In [9]:
def scaling(x):
    sc=MinMaxScaler([0,1])
    x=x.reshape(len(x),len(x[0]),len(x[0][0]),1)
    scaled=np.zeros((len(x),len(x[0]),len(x[0][0]),1))
    #x=x.reshape(len(x),len(x[0]),len(x[0][0]))
    
    for i in range(len(x)):
        for j in range(8):
            sc.fit(x[i][j])
            scaled[i][j]=sc.transform(x[i][j])
            
    return scaled

In [10]:
Xfeat_scaled = scaling(Xfeatures)
Xfeat_scaled = Xfeat_scaled.reshape(-1,8,29)

In [11]:
def wps(x):
    Xps= np.zeros((len(x),len(x[0]),len(x[0][0])))
    
    for i in range(len(x)):
        Xps[i]=np.square(np.abs(x[i]))        
        
    return Xps

In [12]:
Xscaled_power_spectrum = wps(Xfeat_scaled)

In [13]:
Xapprox = Xscaled_power_spectrum[:,0,:]
X_train, X_test, y_train, y_test = train_test_split(Xapprox, train_y, test_size=0.25, random_state=42, stratify=train_y)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=5)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=5) 

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

Number transactions X_train dataset:  (65665, 29)
Number transactions y_train dataset:  (65665, 5)
Number transactions X_test dataset:  (21889, 29)
Number transactions y_test dataset:  (21889, 5)


In [14]:
class InitCentersKMeans(Initializer):
    """ Initializer for initialization of centers of RBF network
        by clustering the given data set.
    # Arguments
        X: matrix, dataset
    """

    def __init__(self, X, max_iter=100):
        self.X = X
        self.max_iter = max_iter
        super().__init__()

    def __call__(self, shape, dtype=None):
        #assert shape[0:] == self.X.shape[1:]

        n_centers = shape[0]
        km = KMeans(n_clusters=n_centers, max_iter=self.max_iter, verbose=0, init='k-means++')
        km.fit(self.X)
        return km.cluster_centers_

In [15]:
class RBFLayer(Layer):

    def __init__(self, output_dim, initializer=None, betas=2.0, **kwargs):

        self.output_dim = output_dim

        # betas is either initializer object or float
        if isinstance(betas, Initializer):
            self.betas_initializer = betas
        else:
            self.betas_initializer = Constant(value=betas)

        self.initializer = initializer if initializer else RandomUniform(
            0.0, 1.0)

        super().__init__(**kwargs)

    def build(self, input_shape):

        self.centers = self.add_weight(name='centers',
                                       shape=(self.output_dim, input_shape[1]),
                                       initializer=self.initializer,
                                       trainable=True)
        self.betas = self.add_weight(name='betas',
                                     shape=(self.output_dim,),
                                     initializer=self.betas_initializer,
                                     # initializer='ones',
                                     trainable=True)

        super().build(input_shape)

    def call(self, x):

        C = tf.expand_dims(self.centers, -1)  # inserts a dimension of 1
        H = tf.transpose(C-tf.transpose(x))  # matrix of differences
        return tf.exp(-self.betas * tf.math.reduce_sum(H**2, axis=1))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        # have to define get_config to be able to use model_from_json
        config = {
            'output_dim': self.output_dim
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [16]:
rbflayer = RBFLayer(232,
                    initializer=InitCentersKMeans(X_train),
                    betas=2.0,
                    input_shape=(29,))
#-----Input layer------#

inputs=tf.keras.layers.Input(shape=(29,))

#-----RBF layer------#

x1=rbflayer(inputs)
x=Dense(5, activation='softmax', name='output')(x1)

outputs=x

model1=tf.keras.Model(inputs=inputs, outputs=outputs)
model1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 29)]              0         
_________________________________________________________________
rbf_layer (RBFLayer)         (None, 232)               6960      
_________________________________________________________________
output (Dense)               (None, 5)                 1165      
Total params: 8,125
Trainable params: 8,125
Non-trainable params: 0
_________________________________________________________________


In [17]:
model1.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer='Adam',
                 metrics=['accuracy'])

annealer = LearningRateScheduler(lambda x: 1e-2 * 0.9 ** x)


history=model1.fit(X_train, y_train,
              batch_size=64,
              epochs=100,
                  steps_per_epoch=len(X_train)//64,
              validation_data=(X_test,y_test),
                  callbacks=[annealer])

Epoch 1/100
1026/1026 [==============================] - 5s 4ms/step - loss: 0.3605 - accuracy: 0.9027 - val_loss: 0.1990 - val_accuracy: 0.9451
Epoch 2/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.2013 - accuracy: 0.9460 - val_loss: 0.1773 - val_accuracy: 0.9549
Epoch 3/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1914 - accuracy: 0.9489 - val_loss: 0.1645 - val_accuracy: 0.9570
Epoch 4/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1703 - accuracy: 0.9546 - val_loss: 0.1578 - val_accuracy: 0.9591
Epoch 5/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1614 - accuracy: 0.9564 - val_loss: 0.1819 - val_accuracy: 0.9497
Epoch 6/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1595 - accuracy: 0.9574 - val_loss: 0.1511 - val_accuracy: 0.9609
Epoch 7/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1516 - accuracy: 0.9590 - val_loss: 0.1450 - val_ac

1026/1026 [==============================] - 3s 3ms/step - loss: 0.1093 - accuracy: 0.9699 - val_loss: 0.1163 - val_accuracy: 0.9686
Epoch 58/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1077 - accuracy: 0.9709 - val_loss: 0.1162 - val_accuracy: 0.9684
Epoch 59/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1149 - accuracy: 0.9680 - val_loss: 0.1161 - val_accuracy: 0.9681
Epoch 60/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1160 - accuracy: 0.9670 - val_loss: 0.1161 - val_accuracy: 0.9683
Epoch 61/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1100 - accuracy: 0.9697 - val_loss: 0.1161 - val_accuracy: 0.9682
Epoch 62/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1073 - accuracy: 0.9699 - val_loss: 0.1162 - val_accuracy: 0.9687
Epoch 63/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1086 - accuracy: 0.9695 - val_loss: 0.1162 - val_accuracy

In [23]:
rbflayer = RBFLayer(580,
                    initializer=InitCentersKMeans(X_train),
                    betas=2.0,
                    input_shape=(29,))
#-----Input layer------#

inputs=tf.keras.layers.Input(shape=(29,))

#-----RBF layer------#

x1=rbflayer(inputs)
x=Dense(5, activation='softmax', name='output')(x1)

outputs=x

model2=tf.keras.Model(inputs=inputs, outputs=outputs)
model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 29)]              0         
_________________________________________________________________
rbf_layer_1 (RBFLayer)       (None, 580)               17400     
_________________________________________________________________
output (Dense)               (None, 5)                 2905      
Total params: 20,305
Trainable params: 20,305
Non-trainable params: 0
_________________________________________________________________


In [24]:
model2.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer='Adam',
                 metrics=['accuracy'])

annealer = LearningRateScheduler(lambda x: 1e-2 * 0.9 ** x)


history=model2.fit(X_train, y_train,
              batch_size=64,
              epochs=100,
                  steps_per_epoch=len(X_train)//64,
              validation_data=(X_test,y_test),
                  callbacks=[annealer])

Epoch 1/100
1026/1026 [==============================] - 4s 3ms/step - loss: 0.3078 - accuracy: 0.9202 - val_loss: 0.1800 - val_accuracy: 0.9559
Epoch 2/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1804 - accuracy: 0.9511 - val_loss: 0.1818 - val_accuracy: 0.9573
Epoch 3/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1688 - accuracy: 0.9544 - val_loss: 0.1883 - val_accuracy: 0.9483
Epoch 4/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1635 - accuracy: 0.9562 - val_loss: 0.1525 - val_accuracy: 0.9605
Epoch 5/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1500 - accuracy: 0.9592 - val_loss: 0.1451 - val_accuracy: 0.9604
Epoch 6/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1464 - accuracy: 0.9603 - val_loss: 0.1471 - val_accuracy: 0.9601
Epoch 7/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.1438 - accuracy: 0.9607 - val_loss: 0.1377 - val_ac

1026/1026 [==============================] - 3s 3ms/step - loss: 0.0951 - accuracy: 0.9727 - val_loss: 0.1031 - val_accuracy: 0.9709
Epoch 58/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.0948 - accuracy: 0.9726 - val_loss: 0.1033 - val_accuracy: 0.9709
Epoch 59/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.0913 - accuracy: 0.9736 - val_loss: 0.1033 - val_accuracy: 0.9709
Epoch 60/100
1026/1026 [==============================] - 4s 3ms/step - loss: 0.0957 - accuracy: 0.9732 - val_loss: 0.1031 - val_accuracy: 0.9709
Epoch 61/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.0907 - accuracy: 0.9740 - val_loss: 0.1031 - val_accuracy: 0.9710
Epoch 62/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.0930 - accuracy: 0.9738 - val_loss: 0.1031 - val_accuracy: 0.9708
Epoch 63/100
1026/1026 [==============================] - 3s 3ms/step - loss: 0.0880 - accuracy: 0.9752 - val_loss: 0.1032 - val_accuracy

In [19]:
Xtest_feats = wpd(test_x)
Xtest_scaled = scaling(Xtest_feats)
Xtest_scaled = Xtest_scaled.reshape(-1,8,29)

Xtest_scaled_power_spectrum = wps(Xtest_scaled)
X_tested = Xtest_scaled_power_spectrum

In [21]:
X_tested_approx = X_tested[:,0,:]
test_y1 = tf.keras.utils.to_categorical(test_y, num_classes=5) 

ypred1 = model1.predict(X_tested_approx)

for i in range(len(ypred1)):
    ind=np.argmax(ypred1[i])
    for j in range(5):
        if j==ind:
            ypred1[i][j]=1.
        else:
            ypred1[i][j]=0.
            


target_names = ['class 0', 'class 1', 'class 2',  'class 3',  'class 4']
print(classification_report(test_y1, ypred1, target_names=target_names, zero_division='warn'))

              precision    recall  f1-score   support

     class 0       0.97      1.00      0.98     18118
     class 1       0.93      0.56      0.70       556
     class 2       0.94      0.87      0.90      1448
     class 3       0.72      0.57      0.64       162
     class 4       0.99      0.91      0.95      1608

   micro avg       0.97      0.97      0.97     21892
   macro avg       0.91      0.78      0.83     21892
weighted avg       0.97      0.97      0.96     21892
 samples avg       0.97      0.97      0.97     21892



In [22]:
model1.evaluate(X_tested_approx, test_y1)

685/685 [==============================] - 2s 2ms/step - loss: 0.1226 - accuracy: 0.9669


[0.12256467342376709, 0.9669285416603088]

In [25]:
X_tested_approx = X_tested[:,0,:]
test_y2 = tf.keras.utils.to_categorical(test_y, num_classes=5) 

ypred2 = model2.predict(X_tested_approx)

for i in range(len(ypred2)):
    ind=np.argmax(ypred2[i])
    for j in range(5):
        if j==ind:
            ypred2[i][j]=1.
        else:
            ypred2[i][j]=0.
            


target_names = ['class 0', 'class 1', 'class 2',  'class 3',  'class 4']
print(classification_report(test_y2, ypred2, target_names=target_names, zero_division='warn'))

              precision    recall  f1-score   support

     class 0       0.97      1.00      0.98     18118
     class 1       0.92      0.58      0.71       556
     class 2       0.94      0.89      0.92      1448
     class 3       0.73      0.56      0.63       162
     class 4       0.99      0.92      0.95      1608

   micro avg       0.97      0.97      0.97     21892
   macro avg       0.91      0.79      0.84     21892
weighted avg       0.97      0.97      0.97     21892
 samples avg       0.97      0.97      0.97     21892



In [26]:
model2.evaluate(X_tested_approx, test_y2)

685/685 [==============================] - 1s 2ms/step - loss: 0.1112 - accuracy: 0.9693


[0.11116894334554672, 0.9693495631217957]